In [1]:
import pandas as pd
import numpy as np

In [36]:
path = '~/Dropbox/208/Project/'
aapl = pd.read_csv(path+'WIKI_PRICES_AAPL.csv')
googl = pd.read_csv(path+'WIKI_PRICES_GOOGL.csv')
fool = pd.read_csv(path+'motley_with_s_scores.csv')

In [37]:
fool.shape

(9959, 6)

In [38]:
fool.head()

,Unnamed: 0,article,date,title,url,sentiment
0,0,Peer-to-peer (P2P) payments have been taking o...,"Jun 6, 2017 at 2:05PM",3 Companies That Could Lose to Apple in P2P Pa...,http://www.fool.com/investing/2017/06/06/3-com...,5
1,1,There are few stocks that have created fortune...,"Jun 6, 2017 at 12:33PM",3 Stocks That Feel Like Microsoft in 1986,http://www.fool.com/investing/2017/06/06/3-sto...,32
2,2,It's been about six months since Apple (NASDAQ...,"Jun 6, 2017 at 10:00AM",Apple Music Is Already a $3.25 Billion Business,http://www.fool.com/investing/2017/06/06/apple...,6
3,3,Bloomberg's Mark Gurman recently published a s...,"Jun 6, 2017 at 9:30AM",What You Need to Know About Apple Inc.'s Artif...,http://www.fool.com/investing/2017/06/06/what-...,10
4,4,"With the iPhone 7-series of smartphones, micro...","Jun 6, 2017 at 8:30AM",What If Apple Inc. Cuts Intel Corporation Out ...,http://www.fool.com/investing/2017/06/06/what-...,2


In [39]:
aapl.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,AAPL,1980-12-12,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.424421,0.426193,0.424421,0.424421,117258400.0
1,AAPL,1980-12-15,27.38,27.38,27.25,27.25,785200.0,0.0,1.0,0.404196,0.404196,0.402277,0.402277,43971200.0
2,AAPL,1980-12-16,25.37,25.37,25.25,25.25,472000.0,0.0,1.0,0.374524,0.374524,0.372752,0.372752,26432000.0
3,AAPL,1980-12-17,25.87,26.00,25.87,25.87,385900.0,0.0,1.0,0.381905,0.383824,0.381905,0.381905,21610400.0
4,AAPL,1980-12-18,26.63,26.75,26.63,26.63,327900.0,0.0,1.0,0.393125,0.394896,0.393125,0.393125,18362400.0


In [41]:
aapl.shape

(9200, 14)

In [42]:
def format_df(stock_df, news_df, word):
    stock_df['diff'] = stock_df['close']-stock_df['open']
    news_df['Count'] = news_df['article'].apply(lambda x: x.count(word))
    news_df.loc[news_df['Count'] <= 5, 'sentiment'] = 0
    news_df['date'] = pd.to_datetime(news_df['date'])
    news_df['date'] = news_df['date'].dt.strftime('%Y-%m-%d')
    news_df = news_df.groupby(['date'], as_index = False).sum()
    news_df = news_df[['date', 'sentiment', 'Count']]
    merged_df = pd.merge(news_df, stock_df)
    merged_df['bin_sentiment'] = pd.cut(merged_df['sentiment'], [-np.inf, -0.001, 0.001, np.inf], labels = [-1, 0, 1])
    merged_df['bin_diff'] = pd.cut(merged_df['diff'], [-np.inf, -0.001, 0.001, np.inf], labels = [-1, 0, 1])
    return merged_df   

In [43]:
merged_df = format_df(aapl, fool, 'Apple')

In [44]:
merged_df.head()

,date,sentiment,Count,ticker,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume,diff,bin_sentiment,bin_diff
0,2014-05-05,53,133,AAPL,590.14,601.0000,590.00,600.9600,10252400.0,0.00,1.0,79.229879,80.687900,79.211083,80.682530,71766800.0,10.8200,1,1
1,2014-05-06,15,177,AAPL,601.80,604.4099,594.41,594.4100,13377300.0,0.00,1.0,80.795305,81.145700,79.803152,79.803152,93641100.0,-7.3900,1,-1
2,2014-05-07,40,134,AAPL,595.25,597.2900,587.73,592.3300,10102300.0,0.00,1.0,79.915927,80.189810,78.906322,79.523900,70716100.0,-2.9200,1,-1
3,2014-05-08,31,109,AAPL,588.25,594.4100,586.40,587.9900,8224900.0,3.29,1.0,79.418033,80.249678,79.168269,79.382931,57574300.0,-0.2600,1,-1
4,2014-05-09,3,67,AAPL,584.54,586.2500,580.33,585.5425,10414200.0,0.00,1.0,78.917156,79.148018,78.348775,79.052500,72899400.0,1.0025,1,1


In [45]:
merged_df.shape

(779, 19)

In [46]:
merged_df.to_csv('merged_df.csv', encoding='utf-8')